<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/05-CSV_Data_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Interacting with a CSV Data

스마트서울 도시데이터 센서(S-DoT) 2분단위 환경정보
더보기
※  데이터 변환으로 2024년 7월 10일 이후 부터 이용 가능합니다.
### 서울시 전역(1100곳)에 설치된 S-DoT 복합 센서를 통해 2분 간격으로 수집되는 기온, 상대습도, 조도, 소음, 진동, 자외선, 풍향, 풍속, 흑구온도, 대기오염물질, 악취 등의 환경정보 데이터입니다.
### 서울 열린데이터광장에서는 동일 데이터로 1시간 단위 평균값으로 제공하고 있습니다.


In [ ]:
!pip install python-dotenv openai langchain langchain-core langchain-community langchain_experimental

In [ ]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
import os
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.0)


## Load the dataset

In [ ]:
import os
import pandas as pd

# Load the Excel file
file_path = 'wv-corelation.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame to understand its structure
df.head()

In [ ]:
# Convert the 'ds' column to datetime format
df['ds'] = pd.to_datetime(df['ds'])

# Set the 'ds' column as the index
df.set_index('ds', inplace=True)

# Resample the data to 1-hour intervals and calculate the mean for each interval
df_resampled = df.resample('1H').mean()

# Display the first few rows of the resampled DataFrame
print(df_resampled.head())
df = df_resampled

## Prepare the Langchain dataframe agent

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Ensure you have your LLM instance created as `llm`

agent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, allow_dangerous_code=True)

# Now you can invoke the agent with a query
response = agent.invoke("what are the names of the columns?")
print(response)

## Design your prompt and ask your question

In [ ]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "Calculate and analyze the correlation of the 'wv' column with the remaining input columns of the 'target variable'."


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)

In [ ]:
QUESTION = "Calculate and analyze the correlation of the 'wv' column with the remaining input columns of the 'target variable' and draw a heatmap."


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)